# ⚖️Creación de un conjunto de datos de preferencia legal en textos juridicos peruanos

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/argilla-io/distilabel/blob/main/docs/tutorials/pipeline-notus-instructions-preferences-legal.ipynb) [![Open Source in Github](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/argilla-io/distilabel/blob/main/docs/tutorials/pipeline-notus-instructions-preferences-legal.ipynb)


Usaremos el modelo mistralai/Mixtral-8x7B-Instruct-v0.1 en endpoints de inferencia para crear un conjunto de datos de preferencia legal basado en las instrucciones RAG de textos juridicos de Perú, en este caso sobre la Constitucion Politica del Perú. Mostraremos cómo usar distilabel para aprovechar los LLM.

[distilabel](https://github.com/argilla-io/distilabel) es un marco de retroalimentación de IA que puede generar y etiquetar conjuntos de datos mediante LLM y puede usarse para muchos casos de uso diferentes. Implementado teniendo en cuenta la solidez, la eficiencia y la escalabilidad, permite a cualquiera crear sus conjuntos de datos sintéticos que pueden usarse en muchos escenarios diferentes. En este notebook crearemos un modelo experto en la Constitucion Politica Peruana, al que podremos realizar diferentes tipos de preguntas y solicitudes.

The LLM model that we will fine-tune for this is [mistralai/Mixtral-8x7B-Instruct-v0.1](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1), una versión mejorada de Zephyr 7B que utiliza Direct Preference Optimization (DPO) y técnicas AIF para superar su modelo básico en varios puntos de referencia y es completamente de código abierto.

Incluiremos los siguientes pasos:

- Definición de una tarea de generación personalizada para un pipeline `distilabel`.
- Creación de un pipeline RAG utilizando Haystack para el documento cargado.
- Generar un conjunto de datos de instrucciones con `SelfInstructTask`.
- Generar un conjunto de datos de preferencias utilizando una tarea de calidad de texto "UltraFeedback".

## Introducción
Comencemos instalando las dependencias necesarias para ejecutar **distilabel** y el resto de los paquetes utilizados en el tutorial; en particular, **Haystack**. Instale también **Argilla** para una mejor visualización y curación de los resultados.

In [4]:
%pip install -q -U distilabel "farm-haystack[preprocessing]"
%pip install -q -U "distilabel[hf-inference-endpoints, argilla]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.8/768.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

### Importar dependencias

Las principales dependencias de este tutorial son distilabel para crear conjuntos de datos sintéticos y Argilla para visualizar y anotar estos conjuntos de datos, y también para ajustar nuestro modelo. El paquete [Haystack](https://haystack.deepset.ai/) se utiliza para crear lotes a partir del documento PDF original desde el que queremos crear nuestros conjuntos de datos.

In [6]:
import os
from typing import Dict

from distilabel.llm import InferenceEndpointsLLM
from distilabel.pipeline import Pipeline, pipeline
from distilabel.tasks import TextGenerationTask, SelfInstructTask, Prompt

from datasets import Dataset

from haystack.nodes import PDFToTextConverter, PreProcessor

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 10>:10                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/haystack/__init__.py:10 in <module>                      │
│                                                                                                  │
│    7                                                                                             │
│    8                                                                                             │
│    9 import haystack.silenceable_tqdm  # Needs to be imported first to wrap TQDM for all foll    │
│ ❱ 10 from haystack.schema import Document, Answer, Label, MultiLabel, Span, EvaluationResult,    │
│   11 from haystack.nodes.base import BaseComponent                                               │
│   12 from haystack.pipelines.base import Pipeline                                                │
│   13 from haystack.environment import set_pytorch_secure_model_loading                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/haystack/schema.py:42 in <module>                        │
│                                                                                                  │
│     39                                                                                           │
│     40                                                                                           │
│     41 @dataclass                                                                                │
│ ❱   42 class Document:                                                                           │
│     43 │   id: str                                                                               │
│     44 │   content: Union[str, DataFrame]                                                        │
│     45 │   content_type: ContentTypes = Field(default="text")                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pydantic/dataclasses.py:250 in dataclass                 │
│                                                                                                  │
│   247 │   if _cls is None:                                                                       │
│   248 │   │   return create_dataclass                                                            │
│   249 │                                                                                          │
│ ❱ 250 │   return create_dataclass(_cls)                                                          │
│   251                                                                                            │
│   252                                                                                            │
│   253 __getattr__ = getattr_migration(__name__)                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pydantic/dataclasses.py:241 in create_dataclass          │
│                                                                                                  │
│   238 │   │   cls.__doc__ = original_doc                                                         │
│   239 │   │   cls.__module__ = original_cls.__module__                                           │
│   240 │   │   cls.__qualname__ = original_cls.__qualname__                                       │
│ ❱ 241 │   │   pydantic_complete = _pydantic_dataclasses.complete_dataclass(                      │
│   242 │   │   │   cls, config_wrapper, raise_errors=False, 

### Variables de entorno

Además, debemos proporcionar nuestro token de acceso HuggingFace u OpenAI si deseas comparar generaciones. Para crear una instancia posterior de un objeto `InferenceEndpointsLLM`, necesitamos pasar como parámetros el nombre del endpoint de inferencia HF y el espacio de nombres HF. Una forma muy conveniente de hacerlo es también mediante variables de entorno.

In [ ]:
os.environ["HF_TOKEN"] = ""
os.environ["HF_INFERENCE_ENDPOINT_NAME"] = "mistralai/Mixtral-8x7B-Instruct-v0.1"
os.environ["HF_NAMESPACE"] = "argilla"
#os.environ["OPENAI_API_KEY"] = ""

## Configuración un punto final de inferencia con Notus

Los endpoints de inferencia son una solución, administrada por Hugging Face, para implementar fácilmente cualquier modelo similar a Transformer. Están construidos a partir de modelos del Hugging Face Hub. Los puntos finales de inferencia son útiles para hacer inferencias en LLM sin la molestia de intentar ejecutar los modelos localmente. En este tutorial, usaremos endpoints de inferencia para generar texto usando nuestro modelo Mistral, como parte del flujo de trabajo "distilabel". El endpoint elegido tiene una instancia Mixtral-8x7B-Instruct-v0.1 en ejecución.

### Definición de una tarea de generación personalizada para una canalización de destilabel

Para iniciar, configuramos un endpoint para nuestro modelo. No es parte del ejemplo de un extremo a otro que veremos más adelante, sino un ejemplo de cómo conectarse a un endpoint de Hugging Face y una prueba de la canalización "distilabel".

Profundicemos en este ejemplo rápido de cómo utilizar un endpoint de inferencia. Hemos preparado una `TextGenerationTask` sencilla para hacerle preguntas al modelo, de una manera muy similar a como hablamos con los LLM usando chatbots. Primero, definimos una clase para la tarea de respuesta a preguntas, con funciones que muestran a `distilabel` cómo el modelo debe generar las indicaciones, analizar la entrada y la salida, etc.

In [ ]:
class QuestionAnsweringTask(TextGenerationTask):
    def generate_prompt(self, question: str) -> str:
        return Prompt(
            system_prompt=self.system_prompt,
            formatted_prompt=question,
        ).format_as(
            "llama2"
        )  # type: ignore

    def parse_output(self, output: str) -> Dict[str, str]:
        return {"answer": output.strip()}

    @property
    def input_args_names(self) -> list[str]:
        return ["question"]

    @property
    def output_args_names(self) -> list[str]:
        return ["answer"]

`llm` es un objeto de la clase `InferenceEndpointsLLM`, y al usarlo podemos comenzar a generar respuestas a preguntas usando el método `llm.generate()`.

In [ ]:
#si estas en colab
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [ ]:
#si estas en jupiter
import getpass
hf_token = getpass.getpass('Enter your password')

Enter your password ········


In [ ]:
llm = InferenceEndpointsLLM(
    endpoint_name_or_model_id=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
    endpoint_namespace=os.getenv("HF_NAMESPACE"),  # type: ignore
    token=hf_token or None,
    task=QuestionAnsweringTask(),
    prompt_format="llama2"
)

INFO:distilabel:Using Serverless Inference Endpoint


Con el objeto `InferenceEndpointsLLM` definido con la información del punto final y la Tarea, podemos continuar y comenzar a generar texto.

In [ ]:
generation = llm.generate(
    [{"question": "Genera un ejemplo de codigo json"}]
)
generation[0][0]["parsed_output"]["answer"]

'Certainly! Here is an example of a JSON (JavaScript Object Notation) code:\n```json\n{\n  "employees": [\n    {\n      "firstName": "John",\n      "lastName": "Doe"\n    },\n    {\n      "firstName": "Anna",\n      "lastName": "Smith"\n    },\n    {\n      "firstName": "Peter",\n      "lastName": "Jones"\n    }\n  ]\n}\n```\nIn this example, the JSON code represents an'

In [ ]:
llm.generate(
    [{"question": "¿Que es la constitucion politica del Perú?"}]
)

[[{'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1',
   'prompt_used': "<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n\n¿Que es la constitucion politica del Perú? [/INST]",
   'raw_output': ' La Constitución Política del Perú es la norma suprema del ordenamiento jurídico de ese país. Es el texto legal que establece las normas, principios y valores que rigen la organización y el funcionamiento del Estado peruano, así como los derechos y deberes de los ciudadanos.\n\nLa Constitución Política del Perú ha

The endpoint is working correctly! We have succesfully set up a custom generating task for a `distilabel` pipeline.


## Creación de un canal RAG utilizando Haystack para la Ley Europea de IA

Para este ejemplo de extremo a extremo, nos gustaría crear un modelo experto capaz de responder preguntas y completar información sobre la Constitucion.

Esta canalización RAG que queremos crear descarga el archivo PDF, lo convierte a texto sin formato y lo preprocesa, creando lotes que podemos alimentar con `dstilabel` para comenzar a crear instrucciones a partir de él. Veamos esta primera parte del proceso y obtengamos los datos de entrada. Tenga en cuenta que esta parte RAG de la canalización no se basa en consultas basadas en una canalización activa o propiedades semánticas, sino en un enfoque más de fuerza bruta en el que descargamos el PDF y preprocesamos su contenido.

### Descarga del PDF

En primer lugar, necesitamos descargar el documento PDF. Lo colocaremos en nuestro directorio de trabajo, si aún no está allí.

In [ ]:
%%bash

if [ ! -f "tuarchivo.pdf" ]; then
    wget -q https://tuenlace.com/tuarchivo.pdf
fi

Una vez que lo tengamos en nuestro directorio de trabajo, podemos usar las funciones Converter y Pipeline de Haystack para extraer los datos textuales, limpiarlos y dividirlos en diferentes lotes. Posteriormente, estos lotes se utilizarán para empezar a crear instrucciones sintéticas.

In [ ]:
%pip install farm-haystack[pdf]==1.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.7/768.7 kB 10.4 MB/s eta 0:00:00
  Using cached pydantic-1.10.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 29.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.4
    Uninstalling pydantic-2.6.4:
      Successfully uninstalled pydantic-2.6.4
  Attempting uninstall: farm-haystack
    Found existing installation: farm-haystack 1.25.2
    Uninstalling farm-haystack-1.25.2:
      Successfully uninstalled farm-haystack-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textdescriptives 2.8.0 requires pydantic>=2.0, but you have pydantic 1.10.15 which is incompatible.


In [ ]:
# OJO
# todo este codigo esta descontinuado, y solo funciona en colab (pero si funciona)
# si te encuentras en jupiter, anaconda, etc usa el codigo debajo de esta seccion

!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.05.tar.gz
!tar -xvf xpdf-tools-linux-4.05.tar.gz && sudo cp xpdf-tools-linux-4.05/bin64/pdftotext /usr/local/bin

# The converter turns the PDF into text we can process easily
converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["es"])

# Preprocessing pipelines can have several steps.
# Ours clean empty lines, header, footers and whitespaces
# and split the text into 150-char long batches, respecting
# where the sentences naturally end and begin.
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=64,
    split_respect_sentence_boundary=True,
)

doc = converter.convert(file_path="constitucion.pdf", meta=None)[0]
docs = preprocessor.process([doc])
print(f"Documents: 1\nBatches: {len(docs)}")


#Let's take a quick look at the batches we just generated.
inputs = [doc.content for doc in docs]
inputs[0][0:256]


### Divisor de PDF en lotes alternativo

In [ ]:
import fitz  # PyMuPDF
import re

def extract_text_from_pdf(file_path):
    """Extrae texto de un archivo PDF."""
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def split_text_by_words(text, max_length, overlap):
    """Divide el texto en lotes por palabras con una longitud máxima y un solapamiento dado."""
    words = re.findall(r'\b[\wáéíóúüÁÉÍÓÚÜ.,;:¡!¿?()"\'\[\]\{\}«»]+\b|\.', text)
    batches = []
    current_batch = ""
    current_length = 0
    for i, word in enumerate(words):
        if current_length + len(word) <= max_length:  # Verifica si la palabra cabe sin superar el límite máximo
            current_batch += word + " "
            current_length += len(word)
        else:
            batches.append(current_batch.strip())
            current_batch = current_batch[-overlap:] + word + " "  # Agrega el solapamiento y la palabra nueva
            current_length = len(current_batch.strip())
    if current_batch:
        batches.append(current_batch.strip())
    return batches

# Ruta al archivo PDF
pdf_file_path = "1constitucion.pdf"

# Extraer texto del PDF
text = extract_text_from_pdf(pdf_file_path)

# Dividir el texto en lotes por palabras con una longitud máxima de 256 y un solapamiento de 128
max_length = 256
overlap = 128
inputs = split_text_by_words(text, max_length, overlap)

# Mostrar el número de lotes
print(f"Documents: 1\nBatches: {len(inputs)}")

# Imprimir algunos lotes para verificar el solapamiento
for i in range(5):
    print(inputs[i])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 34>:34                                                                            │
│ in extract_text_from_pdf:7                                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/fitz/__init__.py:2696 in __init__                        │
│                                                                                                  │
│    2693 │   │   │   if from_file:                                                                │
│    2694 │   │   │   │   if not os.path.exists(filename):                                         │
│    2695 │   │   │   │   │   msg = f"no such file: '{filename}'"                                  │
│ ❱  2696 │   │   │   │   │   raise FileNotFoundError(msg)                                         │
│    2697 │   │   │   │   elif not os.path.isfile(filename):                                       │
│    2698 │   │   │   │   │   msg = f"'{filename}' is no file"                                     │
│    2699 │   │   │   │   │   raise FileDataError(msg)                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: no such file: '1constitucion.pdf'

In [ ]:
# Mostrar el número de lotes
print(f"Documents: 1\nBatches: {len(inputs)}")

# Imprimir el primer lote
print(inputs[0])
print(inputs[1])

El documento se ha agrupado correctamente. Esta lista de cadenas ahora se puede usar como entrada para generar un conjunto de datos de instrucciones usando `dstilabel`.

## Generando instrucciones con SelfInstructTask

Con nuestro punto final de inferencia en funcionamiento, deberíamos poder generar instrucciones con distilabel. Estas instrucciones, realizadas por el LLM a través de nuestro punto final, formarán un conjunto de datos de instrucciones, con instrucciones creadas a partir de los datos que acabamos de extraer.

Para este ejemplo, estamos utilizando un subconjunto de 10 lotes generados en la sección anterior, para cuidar el rendimiento.

In [ ]:
#instructions_dataset = Dataset.from_dict({"input": inputs[0:10]})  #aqui se usan 10 lotes
instructions_dataset = Dataset.from_dict({"input": inputs})         #aqui se usan todos
instructions_dataset

Con la clase `SelfInstructTask` podemos generar una especificación de autoinstrucciones para crear las indicaciones, como se hace en el [documento de autoinstrucciones](https://arxiv.org/abs/2212.10560). `distilabel` comenzará a partir de entradas hechas por humanos, en este caso, los lotes que creamos a partir del pdf, y generará instrucciones basadas en ellos. Luego, estas instrucciones se pueden revisar usando Argilla para conservar las mejores.

Se puede pasar una descripción de la aplicación como parámetro para especificar el comportamiento del modelo; Queremos un modelo capaz de responder a nuestras preguntas sobre la Ley de IA.

In [ ]:
application_description='''An AI assistant specialized in generating questions about the 1993 Constitution of Peru. Users seek its expertise in providing detailed legal insights and expect complete questions for accurate responses. It delves into various legal principles, rights, and government structures comprehensively.
Genera preguntas sobre los articulos de la Constitucion Politica del Peru.
Se espera preguntas completas de los usuarios proporcionando todos los detalles para resolver el problema propuesto o responder preguntas de vulneración de derechos
en los temas de los temas de Educación, Conflictos sociales, Prevención de la corrupción, Servicios públicos, Violencia contra la niñez, Desigualdad y violencia hacia las mujeres, Seguridad ciudadana, Discapacidad o Salud.
Responde unicamente en ESPAÑOL'''

# Por defecto, `SelfInstructTask` generará 5 instrucciones pero se puede modificar este comportamiento con el argumento `num_instructions`.
instruction_task = SelfInstructTask(
    application_description=application_description,
    num_instructions = 4
)

print(f"`SelfInstructTask`\n   - Input arguments: {instruction_task.input_args_names}\n   - Output arguments: {instruction_task.output_args_names}")


`SelfInstructTask`
   - Input arguments: ['input']
   - Output arguments: ['instructions']


Ahora definamos un generador, pasemos el objeto `SelfInstructTask` y creamos un objeto `Pipeline`.

In [ ]:
instructions_generator = InferenceEndpointsLLM(
    endpoint_name_or_model_id=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
    endpoint_namespace=os.getenv("HF_NAMESPACE"),  # type: ignore
    token= hf_token or None,
    task=instruction_task,
    prompt_format="llama2",
)

instructions_pipeline = Pipeline(generator=instructions_generator)

INFO:distilabel:Using Serverless Inference Endpoint


Nuestro pipeline está listo para ser utilizado para generar instrucciones.

In [ ]:
#por defecto el batchsize era 8
generated_instructions = instructions_pipeline.generate(
    dataset=instructions_dataset, num_generations=1, batch_size=16
)

INFO:distilabel:Executing dry-run...
INFO:distilabel:Processing batch 1 of 1...
INFO:distilabel:Calling generator for batch 1...


Flattening the indices:   0%|          | 0/1 [00:00<?, ? examples/s]

INFO:distilabel:Dry-run executed with no issues. Starting the actual generation...
INFO:distilabel:Processing batch 1 of 58...
INFO:distilabel:Calling generator for batch 1...
INFO:distilabel:Processing batch 2 of 58...
INFO:distilabel:Calling generator for batch 2...
INFO:distilabel:Processing batch 3 of 58...
INFO:distilabel:Calling generator for batch 3...
INFO:distilabel:Processing batch 4 of 58...
INFO:distilabel:Calling generator for batch 4...
INFO:distilabel:Processing batch 5 of 58...
INFO:distilabel:Calling generator for batch 5...
INFO:distilabel:Processing batch 6 of 58...
INFO:distilabel:Calling generator for batch 6...
INFO:distilabel:Processing batch 7 of 58...
INFO:distilabel:Calling generator for batch 7...
INFO:distilabel:Processing batch 8 of 58...
INFO:distilabel:Calling generator for batch 8...
INFO:distilabel:Processing batch 9 of 58...
INFO:distilabel:Calling generator for batch 9...
INFO:distilabel:Processing batch 11 of 58...
INFO:distilabel:Calling generator f

Flattening the indices:   0%|          | 0/919 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/919 [00:00<?, ? examples/s]

INFO:distilabel:Checkpoint saved to disk: /content/ckpt.
INFO:distilabel:Final dataset saved at /content/ckpt


El pipeline ha generado instrucciones con éxito teniendo en cuenta los temas y el comportamiento pasado como entrada. Reunamos todas esas instrucciones y veamos cómo se ven.

In [ ]:
import sys
instructions = []
for generations in generated_instructions["instructions"]:
    for generation in generations:
        instructions.extend(generation)

print(f"Number of generated instructions: {len(instructions)}")

for instruction in instructions[:32]:
    print(instruction)

Number of generated instructions: 2076
¿Cuál es el propósito del Preámbulo en la Constitución Política del Perú?
Detalla el mandato del pueblo peruano mencionado en el Preámbulo de la Constitución.
En el contexto de la Constitución Peruana, ¿qué significa obedecer el mandato del pueblo peruano?
Menciona y explique brevemente los temas tratados en el Título I de la Constitución Peruana.
¿Cuál es la importancia de la defensa de la persona humana en la Constitución de 1993 de Perú?
Explica cómo se respeta la dignidad de la persona humana según el artículo 1 de la Constitución peruana de 1993.
Detalle los derechos fundamentales de la persona reconocidos en el Capítulo I del Título I de la Constitución de 1993 de Perú.
¿Cuál es el propósito supremo de la sociedad y el Estado, según el Artículo 1 de la Constitución Política del Perú?
¿Qué derechos inherentes a la persona humana establece el Artículo 2 de la Constitución peruana?
Detalle el alcance y significado de la dignidad humana según la

These initial instructions form our instruction dataset. Following the human-in-the-loop approach, we should push the instructions to Argilla to visualize them and be able to rank them in terms of quality. Those annotations are essential to make quality data, ensuring a better performance of the final model. Nevertheless, this step is optional.


### Subir el conjunto de datos de instrucciones a Argilla para visualizar y anotar.

Echemos un vistazo rápido a las instrucciones generadas por "SelfInstructTask".

In [ ]:
generated_instructions[:1]

{'input': ['CONSTITUCIÓN POLÍTICA DEL PERÚ P R E Á M B U L O EL CONGRESO CONSTITUYENTE DEMOCRÁTICO INVOCANDO A DIOS TODOPODEROSO OBEDECIENDO EL MANDATO DEL PUEBLO PERUANO Y RECORDANDO EL SACRIFICIO DE TODAS LAS GENERACIONES QUE NOS HAN PRECEDIDO EN NUESTRA PATRIA HA RESUELTO DAR LA SIGUIENTE CONSTITUCION TÍTULO I DE'],
 'generation_model': [['mistralai/Mixtral-8x7B-Instruct-v0.1']],
 'generation_prompt': [['<s>[INST] <<SYS>>\nYou are an expert prompt writer, writing the best and most diverse prompts for a variety of tasks. You are given a task description and a set of instructions for how to write the prompts for an specific AI application.<</SYS>>\n\n# Task Description\nDevelop 10 user queries that can be received by the given AI application and applicable to the provided context. Emphasize diversity in verbs and linguistic structures within the model\'s textual capabilities.\n\n# Criteria for Queries\nIncorporate a diverse range of verbs, avoiding repetition.\nEnsure queries are comp

Para cada entrada, es decir, cada lote del archivo pdf, tenemos un mensaje de generador, con pautas generales sobre cómo comportarse, así como el parámetro de descripción de la aplicación. Se han generado 4 instrucciones por entrada.

Ahora es el momento perfecto para cargar el conjunto de datos de instrucciones en Argilla, revisarlo y anotarlo manualmente.

In [ ]:
instructions_rg_dataset = generated_instructions.to_argilla()

<ipython-input-248-ec3f5157e351>:1: UserWarning: Error while converting a row into an Argilla `FeedbackRecord` instance: Skipping the row {'input': 'ey . 8 . A la libertad de creación intelectual artística técnica y científica así como a la propiedad sobre dichas creaciones y a su producto . El Estado propicia el acceso a la cultura y fomenta su desarrollo y difusión . 9 . A la inviolabilidad del domicilio . Nadie puede ingresar en', 'generation_model': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'generation_prompt': '<s>[INST] <<SYS>>\nYou are an expert prompt writer, writing the best and most diverse prompts for a variety of tasks. You are given a task description and a set of instructions for how to write the prompts for an specific AI application.<</SYS>>\n\n# Task Description\nDevelop 10 user queries that can be received by the given AI application and applicable to the provided context. Emphasize diversity in verbs and linguistic structures within the model\'s textual capabilities.\n

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/65.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/65 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/argilla/client/feedback/integrations/textdescriptives.py:72: UserWarning: Using 'en_core_web_md' as default spaCy model for English language.
  warnings.warn("Using 'en_core_web_md' as default spaCy model for English language.")


ℹ The specified spaCy model "en_core_web_md" was not              found
on disk. Downloading...
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.10/dist-packages/textdescriptives/components/coherence.py:44: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  similarities.append(sent.similarity(sents[i + order]))


Output()

In [ ]:
instructions_rg_dataset[0]

FeedbackRecord(fields={'input': 'CONSTITUCIÓN POLÍTICA DEL PERÚ P R E Á M B U L O EL CONGRESO CONSTITUYENTE DEMOCRÁTICO INVOCANDO A DIOS TODOPODEROSO OBEDECIENDO EL MANDATO DEL PUEBLO PERUANO Y RECORDANDO EL SACRIFICIO DE TODAS LAS GENERACIONES QUE NOS HAN PRECEDIDO EN NUESTRA PATRIA HA RESUELTO DAR LA SIGUIENTE CONSTITUCION TÍTULO I DE', 'instructions': '¿Cuál es el propósito del Preámbulo en la Constitución Política del Perú?'}, metadata={'length-input': 305, 'length-instructions': 73, 'generation-model': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'input_n_tokens': 51, 'input_n_unique_tokens': 47, 'input_n_sentences': 7, 'input_perplexity': 1.1, 'input_entropy': 0.09, 'input_flesch_reading_ease': 63.42, 'instructions_n_tokens': 12, 'instructions_n_unique_tokens': 11, 'instructions_n_sentences': 1, 'instructions_perplexity': 1.03, 'instructions_entropy': 0.03, 'instructions_flesch_reading_ease': 74.81}, vectors={'input': [-0.13264846801757812, -0.009939759038388729, 0.12225691974163055, 

In [ ]:
#para este punto ya debes crear el space con argilla en huggingface
%pip install argilla -U
import argilla as rg

In [ ]:
instructions_rg_dataset

FeedbackDataset(
   fields=[TextField(name='input', title='input', required=True, type='text', use_markdown=True), TextField(name='instructions', title='instructions', required=True, type='text', use_markdown=False)]
   questions=[RatingQuestion(name='instruction-rating', title='How would you rate the generated instruction?', description=None, required=True, type='rating', values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])]
   guidelines=None)
   metadata_properties=[IntegerMetadataProperty(name='length-input', title='length-input', visible_for_annotators=True, type='integer', min=None, max=None), IntegerMetadataProperty(name='length-instruction', title='length-instruction', visible_for_annotators=True, type='integer', min=None, max=None), IntegerMetadataProperty(name='input_n_tokens', title='Input N Tokens', visible_for_annotators=True, type='integer', min=None, max=None), IntegerMetadataProperty(name='input_n_unique_tokens', title='Input N Unique Tokens', visible_for_annotators=True, type='inte

In [ ]:
import getpass
argilla_api_key = getpass.getpass('Enter your password')
#argilla_api_key = "admin.apikey"
argilla_space_url = "https://somosnlp-PeruLegalLLMArgilla.hf.space"

Enter your password··········


In [ ]:
rg.init(
    api_key=argilla_api_key,
    api_url=argilla_space_url,
    workspace="adminworkspace"
)

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:195: UserWarning: You're connecting to Argilla Server 1.25.0 using a different client version (1.26.1).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [ ]:
#IMPORTANTE EJECUTAR
instructions_rg_datasett = instructions_rg_dataset

In [ ]:
len(instructions_rg_datasett)

2076

In [ ]:
instructions_rg_datasett

FeedbackDataset(
   fields=[TextField(name='input', title='input', required=True, type='text', use_markdown=True), TextField(name='instructions', title='instructions', required=True, type='text', use_markdown=False)]
   questions=[RatingQuestion(name='instruction-rating', title='How would you rate the generated instruction?', description=None, required=True, type='rating', values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])]
   guidelines=None)
   metadata_properties=[IntegerMetadataProperty(name='length-input', title='length-input', visible_for_annotators=True, type='integer', min=None, max=None), IntegerMetadataProperty(name='length-instruction', title='length-instruction', visible_for_annotators=True, type='integer', min=None, max=None), IntegerMetadataProperty(name='input_n_tokens', title='Input N Tokens', visible_for_annotators=True, type='integer', min=None, max=None), IntegerMetadataProperty(name='input_n_unique_tokens', title='Input N Unique Tokens', visible_for_annotators=True, type='inte

In [ ]:
import argilla as rg

# Agrega la pregunta adicional a la lista de preguntas
questions = [
    rg.RatingQuestion(
        name='instruction-rating',
        title='How would you rate the generated instruction?',
        description=None,
        required=True,
        type='rating',
        values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    ),
    rg.TextQuestion(
        name='curated-instruction',
        title='accurate instruction',
        description='If you think the instruction is not accurate, please correct it.\n If the original instruction is ok, copy and paste it here.',
        required=True,
        type='text',
        use_markdown=False
    )
]


updated_dataset = rg.FeedbackDataset(
    guidelines="The aim of the project is to correct the instructions to make sure they are of the highest quality.",
    fields=instructions_rg_datasett.fields,
    questions=questions,
    metadata_properties=instructions_rg_datasett.metadata_properties,
    vectors_settings=instructions_rg_datasett.vectors_settings
)


updated_dataset.add_records(instructions_rg_datasett.records)
instructions_rg_datasett = updated_dataset

instructions_rg_datasett

FeedbackDataset(
   fields=[TextField(name='input', title='input', required=True, type='text', use_markdown=True), TextField(name='instructions', title='instructions', required=True, type='text', use_markdown=False)]
   questions=[RatingQuestion(name='instruction-rating', title='How would you rate the generated instruction?', description=None, required=True, type=<QuestionTypes.rating: 'rating'>, values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), TextQuestion(name='curated-instruction', title='accurate instruction', description='If you think the instruction is not accurate, please correct it.\n If the original instruction is ok, copy and paste it here.', required=True, type=<QuestionTypes.text: 'text'>, use_markdown=False)]
   guidelines=The aim of the project is to correct the instructions to make sure they are of the highest quality.)
   metadata_properties=[IntegerMetadataProperty(name='length-input', title='length-input', visible_for_annotators=True, type='integer', min=None, max=None), Inte

In [ ]:
instructions_rg_datasett.push_to_argilla(name=f"constitucion_politica_del_peru_1993_q", workspace="adminworkspace")

Output()

INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully pushed to Argilla
INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset(
   id=111f1818-a174-4820-bc0a-251df0f9d18d
   name=constitucion_politica_del_peru_1993_q
   workspace=Workspace(id=156fe7d7-0154-47bd-bf34-d5f97e8ace38, name=adminworkspace, inserted_at=2024-03-20 08:53:04.295557, updated_at=2024-03-20 08:53:04.295557)
   url=https://somosnlp-perulegalllmargilla.hf.space/dataset/111f1818-a174-4820-bc0a-251df0f9d18d/annotation-mode
   fields=[RemoteTextField(id=UUID('8d7b1749-933a-4e47-bb93-c8b3a089ef12'), client=None, name='input', title='input', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('0a620a00-fb31-49f9-96d3-51dd86ec937b'), client=None, name='instructions', title='instructions', required=True, type='text', use_markdown=False)]
   questions=[RemoteRatingQuestion(id=UUID('a389d13d-e6c7-48ac-a01e-e1fd9dfc37cd'), client=None, name='instruction-rating', title='How

RemoteFeedbackDataset(
   id=111f1818-a174-4820-bc0a-251df0f9d18d
   name=constitucion_politica_del_peru_1993_q
   workspace=Workspace(id=156fe7d7-0154-47bd-bf34-d5f97e8ace38, name=adminworkspace, inserted_at=2024-03-20 08:53:04.295557, updated_at=2024-03-20 08:53:04.295557)
   url=https://somosnlp-perulegalllmargilla.hf.space/dataset/111f1818-a174-4820-bc0a-251df0f9d18d/annotation-mode
   fields=[RemoteTextField(id=UUID('8d7b1749-933a-4e47-bb93-c8b3a089ef12'), client=None, name='input', title='input', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('0a620a00-fb31-49f9-96d3-51dd86ec937b'), client=None, name='instructions', title='instructions', required=True, type='text', use_markdown=False)]
   questions=[RemoteRatingQuestion(id=UUID('a389d13d-e6c7-48ac-a01e-e1fd9dfc37cd'), client=None, name='instruction-rating', title='How would you rate the generated instruction?', description=None, required=True, type='rating', values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), Remote

En la interfaz de usuario de Argilla, cada instrucción de entrada de se visualiza individualmente y se puede anotar individualmente.


##  Extraccion de dataset despues de realizar el feedback


In [ ]:
import argilla as rg

#extraigo el feedback dataset de argilla
dataset = rg.FeedbackDataset.from_argilla(
    "constitucion_politica_del_peru_1993_q", workspace="adminworkspace"
)

In [ ]:
dataset[0]

RemoteFeedbackRecord(id=UUID('6c954d76-be27-42e7-883b-2a6e2c584c1e'), client=<httpx.Client object at 0x7d30580d5090>, fields={'input': 'CONSTITUCIÓN POLÍTICA DEL PERÚ P R E Á M B U L O EL CONGRESO CONSTITUYENTE DEMOCRÁTICO INVOCANDO A DIOS TODOPODEROSO OBEDECIENDO EL MANDATO DEL PUEBLO PERUANO Y RECORDANDO EL SACRIFICIO DE TODAS LAS GENERACIONES QUE NOS HAN PRECEDIDO EN NUESTRA PATRIA HA RESUELTO DAR LA SIGUIENTE CONSTITUCION TÍTULO I DE', 'instructions': '¿Cuál es el propósito del Preámbulo en la Constitución Política del Perú?'}, metadata={'length-input': 305, 'length-instructions': 73, 'generation-model': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'input_n_tokens': 51, 'input_n_unique_tokens': 47, 'input_n_sentences': 7, 'input_perplexity': 1.1, 'input_entropy': 0.09, 'input_flesch_reading_ease': 63.42, 'instructions_n_tokens': 12, 'instructions_n_unique_tokens': 11, 'instructions_n_sentences': 1, 'instructions_perplexity': 1.03, 'instructions_entropy': 0.03, 'instructions_flesch_readi

## Subo el dataset a huggingface

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#este es el dataset con la estructura de argilla
dataset.push_to_huggingface("daqc/constitucion_politica_del_peru_1993_q_argilla")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
#yo recomiendo subir el dataset que hicimos al comienzo con la estructura de llama2 con [INST] [/INST] (por si en algun momento lo necesitas)
generated_instructions.push_to_hub("daqc/constitucion_politica_del_peru_1993_q_raw")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

INFO:distilabel:Pushing task to the hub...


## **Genere un conjunto de datos de preferencia mediante una tarea de calidad de texto Ultrafeedback.**

Una vez que tengamos nuestro conjunto de datos de instrucciones, crearemos un conjunto de datos de preferencias a través de la tarea de calidad del texto UltraFeedback. Este es un tipo de tarea utilizada en PNL que se utiliza para evaluar la calidad del texto generado; Nuestro objetivo es proporcionar comentarios detallados sobre la calidad del texto generado, más allá de una etiqueta binaria.

Nuestro método `pipeline()` nos permite crear una instancia de `Pipeline` con los LLM proporcionados para una tarea determinada, lo cual es útil siempre que desee utilizar un `Pipeline` predefinido o personalizado para una tarea determinada. Especificaremos nuestra tarea y subtarea, el generador que queremos usar (en este caso, uno basado en una Tarea de Generador de Texto) y nuestra clave API OpenAI.

In [ ]:
# NO USARRRR
#esto es en caso hayas agregado tu api de openai para generar la respuestas a las preguntas que ya tenemos
#asi que aqui no sirveeeee
preference_pipeline = pipeline(
    "preference",
    "instruction-following",
    generator=InferenceEndpointsLLM(
        endpoint_name_or_model_id=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
        endpoint_namespace=os.getenv("HF_NAMESPACE", None),
        task=TextGenerationTask(),
        max_new_tokens=256,
        num_threads=2,
        temperature=0.3,
    ),
    max_new_tokens=256,
    num_threads=2,
    api_key=os.getenv("OPENAI_API_KEY", None),
    temperature=0.0,
)

Definimos el prompt para el endpoint

In [ ]:
application_description='''An AI assistant specialized in answering questions and resolving vulnerabilty right scenarios with only the 1993 Constitution of Peru.
Users seek its expertise in providing detailed legal insights and expect complete responses. It delves into various legal principles, rights, and government structures comprehensively.
Respondes solo en formato JSON y respeta las instrucciones para las siguientes claves, IMPORTANTE NO CREAR SUBCLAVES, toda la respuesta debe ir dentro de la clave 'respuesta':
1. respuesta: Solo en esta seccion se detallara toda, TODA, TODA la respuesta, resolver el problema propuesto o brindar solucion a situaciones de vulneración de derechos.
2. fuente: La fuente debe ser siempre 'Constitución Política del Perú 1993', variando únicamente el número de artículo y capítulo. Solamente debes basarte en los articulos de la Constitucion Politica del Peru de 1993, puedes usar otro texto juridico solo en casos extremos, en este caso se debera separar los textos juridicos con sus respectivos capitulos y articulos con punto y coma.
3. tema: No inventes ni crees un nuevo tema, en caso de considerarse mas de 2 temas se separa con punto y coma, escoge solo los que se adecuen a la respuesta, no consideres todos al mismo tiempo: Educación, Conflictos sociales, Prevención de la corrupción, Servicios públicos, Violencia contra la niñez, Desigualdad y violencia hacia las mujeres, Seguridad ciudadana, Discapacidad o Salud.

Respondes unicamente en ESPAÑOL en base al contexto y la pregunta, y solo en formato JSON sin agregar alguna clave adicional o extra, PROHIBIDO CREAR SUBCLAVES, UNICAMENTE SE TENDRAN 3, guiate en el siguiente ejemplo:
{
  "respuesta": "La defensa de la persona humana y el respeto de su dignidad son el fin supremo de la sociedad y del Estado.", "fuente": "Constitución Política del Perú 1993 - Capitulo 1, Artículo 1;", "tema": "Violencia contra la niñez; Desigualdad y violencia hacia las mujeres; Seguridad ciudadana"
}
NO DEBE GENERARSE TEXTO DESPUES DE MENCIONAR LOS TEMAS ESCOGIDOS DE LA LISTA PROPORCIONADA.
'''


text_generation_task = TextGenerationTask(system_prompt=application_description)
text_generation_task

TextGenerationTask(system_prompt='An AI assistant specialized in answering questions and resolving vulnerabilty right scenarios with only the 1993 Constitution of Peru.\nUsers seek its expertise in providing detailed legal insights and expect complete responses. It delves into various legal principles, rights, and government structures comprehensively.\nRespondes solo en formato JSON y respeta las instrucciones para las siguientes claves, IMPORTANTE NO CREAR SUBCLAVES, toda la respuesta debe ir dentro de la clave \'respuesta\':\n1. respuesta: Solo en esta seccion se detallara toda, TODA, TODA la respuesta, resolver el problema propuesto o brindar solucion a situaciones de vulneración de derechos.\n2. fuente: La fuente debe ser siempre \'Constitución Política del Perú 1993\', variando únicamente el número de artículo y capítulo. Solamente debes basarte en los articulos de la Constitucion Politica del Peru de 1993, puedes usar otro texto juridico solo en casos extremos, en este caso se d

En la forma de llamar a nuestra pipeline, tendremos una nueva Task, y dado que los problemas pueden requerir mayor cantidad de texto, vamos a modificar max_new_tokens a 1024.

In [ ]:
preference_pipeline = Pipeline(
    generator=InferenceEndpointsLLM(
        endpoint_name_or_model_id=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
        endpoint_namespace=os.getenv("HF_NAMESPACE", None),
        #task=TextGenerationTask(),
        task=text_generation_task,
        token=hf_token,
        prompt_format="llama2",
        max_new_tokens=2048,
        num_threads=4,
        temperature=0.3
    )
)

INFO:distilabel:Using Serverless Inference Endpoint


## Nos aseguramos que tenemos listo nuestro conjunto de datos de instrucciones, ya que será la entrada de este pipeline. (puedes hacerlo de 2 formas)

## **1 forma: Haciendo pull del feedback dataset que tienes en argilla**  (Hazlo con este)

In [ ]:
import getpass
import argilla as rg
argilla_api_key = getpass.getpass('Enter your password')
#argilla_api_key = "admin.apikey"
argilla_space_url = "https://somosnlp-PeruLegalLLMArgilla.hf.space"

rg.init(
    api_key=argilla_api_key,
    api_url=argilla_space_url,
    workspace="adminworkspace"
)

Enter your password··········


In [ ]:

#extraigo el feedback dataset de argilla y convierto a dataset normal para continuar
remote_dataset = rg.FeedbackDataset.from_argilla(
    "constitucion_politica_del_peru_1993_q", workspace="adminworkspace"
)

In [ ]:
#debes convertir a dataset normal para continuar
#instructions_dataset = remote_dataset.pull(max_records=5)  # get first 10 records
instructions_dataset = remote_dataset.pull()
#
instructions_dataset = instructions_dataset.format_as("datasets")
instructions_dataset

Dataset({
    features: ['input', 'instructions', 'instruction-rating', 'instruction-rating-suggestion', 'instruction-rating-suggestion-metadata', 'curated-instruction', 'curated-instruction-suggestion', 'curated-instruction-suggestion-metadata', 'external_id', 'metadata', 'vectors'],
    num_rows: 2075
})

Antes de generar el texto según nuestras instrucciones, debemos mezclarnos un poco con el conjunto de datos. De la sección anterior, todavía tenemos nuestra entrada anterior, los lotes del PDF. Tenemos que cambiar eso a las instrucciones que generamos.

In [ ]:
#IMPORTANTE EJECUTAR ESTEEE SOLO UNA VEZ
instructions_dataset = instructions_dataset.rename_columns({"input": "context", "instructions": "input"})

In [ ]:
instructions_dataset

Dataset({
    features: ['context', 'input', 'instruction-rating', 'instruction-rating-suggestion', 'instruction-rating-suggestion-metadata', 'curated-instruction', 'curated-instruction-suggestion', 'curated-instruction-suggestion-metadata', 'external_id', 'metadata', 'vectors'],
    num_rows: 2075
})

In [ ]:
instructions_dataset[0]

{'context': 'CONSTITUCIÓN POLÍTICA DEL PERÚ P R E Á M B U L O EL CONGRESO CONSTITUYENTE DEMOCRÁTICO INVOCANDO A DIOS TODOPODEROSO OBEDECIENDO EL MANDATO DEL PUEBLO PERUANO Y RECORDANDO EL SACRIFICIO DE TODAS LAS GENERACIONES QUE NOS HAN PRECEDIDO EN NUESTRA PATRIA HA RESUELTO DAR LA SIGUIENTE CONSTITUCION TÍTULO I DE',
 'input': '¿Cuál es el propósito del Preámbulo en la Constitución Política del Perú?',
 'instruction-rating': [],
 'instruction-rating-suggestion': None,
 'instruction-rating-suggestion-metadata': {'type': None,
  'score': None,
  'agent': None},
 'curated-instruction': [],
 'curated-instruction-suggestion': None,
 'curated-instruction-suggestion-metadata': {'type': None,
  'score': None,
  'agent': None},
 'external_id': None,
 'metadata': '{"length-input": 305, "length-instructions": 73, "generation-model": "mistralai/Mixtral-8x7B-Instruct-v0.1", "input_n_tokens": 51, "input_n_unique_tokens": 47, "input_n_sentences": 7, "input_perplexity": 1.1, "input_entropy": 0.09, "

## **2 forma: Haciendo pull del dataset que subimos a hg con  la estructura de llama2** (este NO lo probe)

El dataset debe quedar asi (ya lo tenemos guardado en la variable 'generated_instructions' si ejecutaste los anteriores pasos, sino carga el dataset desde tu cuenta hg):

In [ ]:
#generated_instructions[0]

A continuación vamos a transformar nuestro dataset con instrucciones al formato esperado por distilabel para la generación, extrayendo todas las instrucciones anidadas, y poniendo la columna "input".

In [ ]:
#rows = []
#from datasets import Dataset

#generations = []
#for row in generated_instructions:
#    for instructions in row["instructions"]:
#        for generation in instructions:
#            generations.append(generation)
#
#generation_dataset = Dataset.from_dict({"input": generations})

## GENERAMOS EL DATASET DE RESPUESTAS

Ahora, creemos un conjunto de datos utilizando la canalización que acabamos de crear y los temas a partir de los cuales se generaron nuestras instrucciones.

In [ ]:
#reemplaza instructions_dataset por generation_dataset o vicerversa segun la forma que hayas escogido
# en este caso yo hice pull del feedbackdataset de argilla (forma 1), por lo que uso instructions_dataset
#en la variable num_generations pondria 2 si utilizara mi apikey de openai para comparar generaciones de respuestas
preference_dataset = preference_pipeline.generate(
    instructions_dataset,  # type: ignore
    num_generations=1,
    batch_size=32,  #16
    display_progress_bar=True,
)

INFO:distilabel:Executing dry-run...
INFO:distilabel:Processing batch 1 of 1...
INFO:distilabel:Calling generator for batch 1...


Flattening the indices:   0%|          | 0/1 [00:00<?, ? examples/s]

INFO:distilabel:Dry-run executed with no issues. Starting the actual generation...
INFO:distilabel:Processing batch 1 of 65...
INFO:distilabel:Calling generator for batch 1...
INFO:distilabel:Processing batch 2 of 65...
INFO:distilabel:Calling generator for batch 2...
INFO:distilabel:Processing batch 3 of 65...
INFO:distilabel:Calling generator for batch 3...
INFO:distilabel:Processing batch 4 of 65...
INFO:distilabel:Calling generator for batch 4...
INFO:distilabel:Processing batch 5 of 65...
INFO:distilabel:Calling generator for batch 5...
INFO:distilabel:Processing batch 6 of 65...
INFO:distilabel:Calling generator for batch 6...
INFO:distilabel:Processing batch 7 of 65...
INFO:distilabel:Calling generator for batch 7...
INFO:distilabel:Processing batch 8 of 65...
INFO:distilabel:Calling generator for batch 8...
INFO:distilabel:Processing batch 9 of 65...
INFO:distilabel:Calling generator for batch 9...
INFO:distilabel:Processing batch 10 of 65...
INFO:distilabel:Calling generator f

Flattening the indices:   0%|          | 0/2075 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2075 [00:00<?, ? examples/s]

INFO:distilabel:Checkpoint saved to disk: /content/ckpt.
INFO:distilabel:Final dataset saved at /content/ckpt


Echemos un vistazo a una instancia del conjunto de datos de preferencias:

In [ ]:
preference_dataset[2004]

{'context': 'nte . Duodécima . La organización política departamental de la República comprende los departamentos siguientes Amazonas Ancash Apurímac Arequipa Ayacucho Cajamarca Cusco Huancavelica Huánuco Ica Junín La Libertad Lambayeque Lima Loreto Madre de Dios Moquegua Pasco Piura Puno',
 'input': 'Detalle las funciones y responsabilidades del departamento de Amazonas en la organización política peruana.',
 'instruction-rating': [],
 'instruction-rating-suggestion': None,
 'instruction-rating-suggestion-metadata': {'agent': None,
  'score': None,
  'type': None},
 'curated-instruction': [],
 'curated-instruction-suggestion': None,
 'curated-instruction-suggestion-metadata': {'agent': None,
  'score': None,
  'type': None},
 'external_id': None,
 'metadata': '{"length-input": 276, "length-instructions": 107, "generation-model": "mistralai/Mixtral-8x7B-Instruct-v0.1", "input_n_tokens": 36, "input_n_unique_tokens": 33, "input_n_sentences": 5, "input_perplexity": 1.34, "input_entropy": 

In [ ]:
preference_dataset

Dataset({
    features: ['context', 'input', 'instruction-rating', 'instruction-rating-suggestion', 'instruction-rating-suggestion-metadata', 'curated-instruction', 'curated-instruction-suggestion', 'curated-instruction-suggestion-metadata', 'external_id', 'metadata', 'vectors', 'generation_model', 'generation_prompt', 'raw_generation_responses', 'generations'],
    num_rows: 2075
})

In [ ]:
preference_dataset2 = preference_dataset

## Comentarios humanos con Argilla

Puede utilizar los comentarios de IA creados por distilabel directamente, pero hemos visto que mejorarlos con comentarios humanos mejorará la calidad de su LLM. Proporcionamos un método `to_argilla` que crea un conjunto de datos para Argilla junto con filtros de metadatos personalizados y búsqueda semántica listos para usar para permitirle brindar comentarios humanos de la manera más rápida y atractiva posible. Puede consultar [los documentos de Argilla](https://docs.argilla.io/en/latest/getting_started/quickstart_installation.html) para ponerlo en funcionamiento.

Si está ejecutando Argilla usando la imagen de inicio rápido de Docker o Hugging Face Spaces, debe iniciar el cliente Argilla con la URL y API_KEY:

In [ ]:
import argilla as rg

# Replace api_url with the url to your HF Spaces URL if using Spaces
# Replace api_key if you configured a custom API key
argilla_api_key = getpass.getpass('Enter your password')
argilla_space_url = "https://somosnlp-PeruLegalLLMArgilla.hf.space"


rg.init(
    api_key=argilla_api_key,
    api_url=argilla_space_url,
    workspace="adminworkspace"
)

Enter your password··········


/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:195: UserWarning: You're connecting to Argilla Server 1.25.0 using a different client version (1.26.1).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


Realizamos la conversion a formato argilla y lo subimos

In [ ]:
# Uploading the Preference Dataset
preference_rg_dataset = preference_dataset.to_argilla()

Adding text descriptives to metadata... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

In [ ]:
preference_rg_dataset

FeedbackDataset(
   fields=[TextField(name='input', title='input', required=True, type='text', use_markdown=True), TextField(name='generations-1', title='generations-1', required=True, type='text', use_markdown=True)]
   questions=[RatingQuestion(name='generations-1-rating', title='How would you rate the generation at `generations-1`?', description=None, required=True, type='rating', values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])]
   guidelines=None)
   metadata_properties=[IntegerMetadataProperty(name='length-input', title='length-input', visible_for_annotators=True, type='integer', min=None, max=None), IntegerMetadataProperty(name='length-generations-1', title='length-generations-1', visible_for_annotators=True, type='integer', min=None, max=None), IntegerMetadataProperty(name='input_n_tokens', title='Input N Tokens', visible_for_annotators=True, type='integer', min=None, max=None), IntegerMetadataProperty(name='input_n_unique_tokens', title='Input N Unique Tokens', visible_for_annotators=T

In [ ]:
# Importa el módulo necesario para modificar el dataset
import argilla as rg

# Agrega la pregunta adicional a la lista de preguntas
questions = [
    rg.RatingQuestion(
        name='generations-1-rating',
        title= 'How would you rate the generation at `generations-1`?',
        description=None,
        required=True,
        type='rating',
        values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    ),
    rg.TextQuestion(
        name='curated-generation',
        title='accurate generation',
        description='If you think the generation is not accurate, please correct it.\n If the original generation is ok, copy and paste it here.',
        required=True,
        type='text',
        use_markdown=False
    )
]


updated_dataset2 = rg.FeedbackDataset(
    guidelines="The aim of the project is to correct the generations to make sure they are of the highest quality.",
    fields=preference_rg_dataset.fields,
    questions=questions,
    metadata_properties=preference_rg_dataset.metadata_properties,
    vectors_settings=preference_rg_dataset.vectors_settings
)


# Agrega los registros originales al nuevo dataset
updated_dataset2.add_records(preference_rg_dataset.records)
#volvemos a la variable que ya estaba usando en todo el codigo y me da flojera cambiar a updated_dataset2
preference_rg_dataset = updated_dataset2

preference_rg_dataset

FeedbackDataset(
   fields=[TextField(name='input', title='input', required=True, type='text', use_markdown=True), TextField(name='generations-1', title='generations-1', required=True, type='text', use_markdown=True)]
   questions=[RatingQuestion(name='generations-1-rating', title='How would you rate the generation at `generations-1`?', description=None, required=True, type=<QuestionTypes.rating: 'rating'>, values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), TextQuestion(name='curated-generation', title='accurate generation', description='If you think the generation is not accurate, please correct it.\n If the original generation is ok, copy and paste it here.', required=True, type=<QuestionTypes.text: 'text'>, use_markdown=False)]
   guidelines=The aim of the project is to correct the generations to make sure they are of the highest quality.)
   metadata_properties=[IntegerMetadataProperty(name='length-input', title='length-input', visible_for_annotators=True, type='integer', min=None, max=None)

In [ ]:

# Adding the context as a metadata property in the new Feedback dataset, as this
# information will be useful later.
for record_feedback, record_huggingface in zip(
    preference_rg_dataset, preference_dataset
):
    record_feedback.metadata["context"] = record_huggingface["context"]

preference_rg_dataset.push_to_argilla(name=f"constitucion_politica_del_peru_1993_qa", workspace="adminworkspace")

Output()

INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully pushed to Argilla
INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset(
   id=0fb0ab8f-94e2-4958-8aba-4403993a3372
   name=constitucion_politica_del_peru_1993_qa
   workspace=Workspace(id=156fe7d7-0154-47bd-bf34-d5f97e8ace38, name=adminworkspace, inserted_at=2024-03-20 08:53:04.295557, updated_at=2024-03-20 08:53:04.295557)
   url=https://somosnlp-perulegalllmargilla.hf.space/dataset/0fb0ab8f-94e2-4958-8aba-4403993a3372/annotation-mode
   fields=[RemoteTextField(id=UUID('0925d12a-99e2-4887-b150-934b4fd77619'), client=None, name='input', title='input', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('8bb31404-9d99-4197-b1af-5c62de05d1fe'), client=None, name='generations-1', title='generations-1', required=True, type='text', use_markdown=True)]
   questions=[RemoteRatingQuestion(id=UUID('025c575a-abc4-464c-8db6-83e1e8289f42'), client=None, name='generations-1-rating', title=

RemoteFeedbackDataset(
   id=0fb0ab8f-94e2-4958-8aba-4403993a3372
   name=constitucion_politica_del_peru_1993_qa
   workspace=Workspace(id=156fe7d7-0154-47bd-bf34-d5f97e8ace38, name=adminworkspace, inserted_at=2024-03-20 08:53:04.295557, updated_at=2024-03-20 08:53:04.295557)
   url=https://somosnlp-perulegalllmargilla.hf.space/dataset/0fb0ab8f-94e2-4958-8aba-4403993a3372/annotation-mode
   fields=[RemoteTextField(id=UUID('0925d12a-99e2-4887-b150-934b4fd77619'), client=None, name='input', title='input', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('8bb31404-9d99-4197-b1af-5c62de05d1fe'), client=None, name='generations-1', title='generations-1', required=True, type='text', use_markdown=True)]
   questions=[RemoteRatingQuestion(id=UUID('025c575a-abc4-464c-8db6-83e1e8289f42'), client=None, name='generations-1-rating', title='How would you rate the generation at `generations-1`?', description=None, required=True, type='rating', values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 

In [ ]:
preference_rg_dataset[0]

En la interfaz de usuario de Argilla, podemos ver la entrada (una instrucción) y las dos generaciones que el LLM creó a partir de ella.

## Subida del dataset a huggingface

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#este es el dataset con la estructura de argilla (contiene las preguntas y respuestas)
#extraigo el feedback dataset de argilla
dataset2 = rg.FeedbackDataset.from_argilla(
    "constitucion_politica_del_peru_1993_qa", workspace="adminworkspace"
)

dataset2[0:1]

[RemoteFeedbackRecord(id=UUID('30b46291-02ca-4f62-a506-4825c98391db'), client=<httpx.Client object at 0x78699b59f430>, fields={'input': '¿Cuál es el propósito del Preámbulo en la Constitución Política del Perú?', 'generations-1': '{\n  "pregunta": "¿Cuál es el propósito del Preámbulo en la Constitución Política del Perú?",\n  "respuesta": "El Preámbulo de la Constitución Política del Perú establece el objetivo de refundar la República para lograr una sociedad democrática, justa y libre, con pleno respeto a los derechos humanos, así como el fortalecimiento del Estado de Derecho para asegurar el desarrollo integral, la justicia social y la solidaridad entre los peruanos.",\n  "fuente": "Constitución Política del Perú 1993 - Preámbulo;",\n  "tema": "Educación; Prevención de la corrupción; Servicios públicos; Desigualdad y violencia hacia las mujeres; Seguridad ciudadana; Discapacidad; Salud"\n}'}, metadata={'length-input': 73, 'length-generations-1': 668, 'generation-model': ['mistralai/M

In [ ]:
#se sube el dataset con la estructura de argilla para persistencia :c
dataset2.push_to_huggingface("daqc/constitucion_politica_del_peru_1993_qa_argilla")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
#se sube el dataset con la estructura de llama2 con [INST] [/INST] para el entrenamiento (contiene las preguntas y respuestas)
preference_dataset.push_to_hub("daqc/constitucion_politica_del_peru_1993_qa_raw")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

INFO:distilabel:Pushing task to the hub...


## Conclusiones

Para concluir, hemos repasado un ejemplo de destilabel de un extremo a otro. Configuramos un  endpoint de inferencia, definimos un pipeline de etiquetas que extrae información de un PDF y creamos y revisamos manualmente el conjunto de datos de instrucciones y preferencias creado a partir de esa entrada. El conjunto de datos de preferencias final es perfecto para realizar ajustes, y puede hacerlo fácilmente utilizando ArgillaTrainer de Argilla. Echa un vistazo a estos recursos si quieres ir más allá:

- [Entrenar un modelo con ArgillaTrainer](https://docs.argilla.io/en/latest/tutorials_and_integrations/tutorials/feedback/end2end_examples/train-model-006.html)
- [Ⓜ️ Ajuste de LLM como asistentes de chat: Ajuste supervisado en Mistral 7B](https://docs.argilla.io/en/latest/tutorials_and_integrations/tutorials/feedback/training-llm-mistral-sft.html)
- [🌠 Mejora de RAG optimizando los modelos de recuperación y reclasificación](https://docs.argilla.io/en/latest/tutorials_and_integrations/tutorials/feedback/fine-tuning-sentencesimilarity-rag.html)

# LMIPIEZA DE DATOS

# Conversion del dataset para el entrenamiento en formato gemma 2b it

In [ ]:
dataset2

RemoteFeedbackDataset(
   id=0fb0ab8f-94e2-4958-8aba-4403993a3372
   name=constitucion_politica_del_peru_1993_qa
   workspace=Workspace(id=156fe7d7-0154-47bd-bf34-d5f97e8ace38, name=adminworkspace, inserted_at=2024-03-20 08:53:04.295557, updated_at=2024-03-20 08:53:04.295557)
   url=https://somosnlp-perulegalllmargilla.hf.space/dataset/0fb0ab8f-94e2-4958-8aba-4403993a3372/annotation-mode
   fields=[RemoteTextField(id=UUID('0925d12a-99e2-4887-b150-934b4fd77619'), client=None, name='input', title='input', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('8bb31404-9d99-4197-b1af-5c62de05d1fe'), client=None, name='generations-1', title='generations-1', required=True, type='text', use_markdown=True)]
   questions=[RemoteRatingQuestion(id=UUID('025c575a-abc4-464c-8db6-83e1e8289f42'), client=None, name='generations-1-rating', title='How would you rate the generation at `generations-1`?', description=None, required=True, type='rating', values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 

In [ ]:
# Convertimos de argilla a formato dataset
dataset2_converted = dataset2.format_as("datasets")

In [ ]:
dataset2_converted

Dataset({
    features: ['input', 'generations-1', 'generations-1-rating', 'generations-1-rating-suggestion', 'generations-1-rating-suggestion-metadata', 'curated-generation', 'curated-generation-suggestion', 'curated-generation-suggestion-metadata', 'external_id', 'metadata', 'vectors'],
    num_rows: 2075
})

In [ ]:
dataset2_converted[0]

{'input': '¿Cuál es el propósito del Preámbulo en la Constitución Política del Perú?',
 'generations-1': '{\n  "pregunta": "¿Cuál es el propósito del Preámbulo en la Constitución Política del Perú?",\n  "respuesta": "El Preámbulo de la Constitución Política del Perú establece el objetivo de refundar la República para lograr una sociedad democrática, justa y libre, con pleno respeto a los derechos humanos, así como el fortalecimiento del Estado de Derecho para asegurar el desarrollo integral, la justicia social y la solidaridad entre los peruanos.",\n  "fuente": "Constitución Política del Perú 1993 - Preámbulo;",\n  "tema": "Educación; Prevención de la corrupción; Servicios públicos; Desigualdad y violencia hacia las mujeres; Seguridad ciudadana; Discapacidad; Salud"\n}',
 'generations-1-rating': [],
 'generations-1-rating-suggestion': None,
 'generations-1-rating-suggestion-metadata': {'type': None,
  'score': None,
  'agent': None},
 'curated-generation': [],
 'curated-generation-sugg

In [ ]:
from datasets import Dataset

# Crear un objeto Dataset a partir de tus datos
new_dataset = Dataset.from_dict({'input': [entry['input'] for entry in dataset2_converted],
                                 'generations-1': [entry['generations-1'] for entry in dataset2_converted]})
#
# Cambiar los nombres de las características a INSTRUCTION y RESPONSE
new_dataset = new_dataset.rename_column("input", "pregunta")
new_dataset = new_dataset.rename_column("generations-1", "respuesta")

In [ ]:
# Verificar la descripción del nuevo dataset
new_dataset

In [ ]:
new_dataset[0]

In [ ]:
new_dataset.push_to_hub("daqc/constitucion-politica-del-peru-1993-qa-gemma-2b-it-format")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/315 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/daqc/constitucion-politica-del-peru-1993-qa-gemma-2b-it-format/commit/69965238ae7e1058b14a949ad91089110b8f2410', commit_message='Upload dataset', commit_description='', oid='69965238ae7e1058b14a949ad91089110b8f2410', pr_url=None, pr_revision=None, pr_num=None)

# Dividimos el dataset (80% train / 20% test)

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
from datasets import load_dataset, DatasetDict

# Cargar el dataset desde Hugging Face
divided_dataset_dict = load_dataset("daqc/constitucion-politica-del-peru-1993-qa-gemma-2b-it-format")
# Dividir el dataset en partes de entrenamiento y prueba (por ejemplo, 80% train, 20% test)
train_dataset = divided_dataset_dict['train'].train_test_split(test_size=0.2,  shuffle=True, seed=42)
# Crear un nuevo DatasetDict con las partes divididas
new_divided_dataset_dict = DatasetDict({'train': train_dataset['train'], 'test': train_dataset['test']})
# Subir el dataset combinado a Hugging Face
new_divided_dataset_dict.push_to_hub("daqc/constitucion-politica-del-peru-1993-qa-gemma-2b-it-format-80train-20test")

Generating train split:   0%|          | 0/2075 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/daqc/constitucion-politica-del-peru-1993-qa-gemma-2b-it-format-80train-20test/commit/0f208f35cfb457df3a033c6f53aae5a5d50455ea', commit_message='Upload dataset', commit_description='', oid='0f208f35cfb457df3a033c6f53aae5a5d50455ea', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
new_divided_dataset_dict

# LIMPIEZA TOTAL PARA PUBLICAR

from datasets import load_dataset

# Cargar el conjunto de datos
dataset = load_dataset("daqc/constitucion-politica-del-peru-1993-qa-gemma-2b-it-format-80train-20test")

# Verificar las claves disponibles en el conjunto de datos
print(dataset.keys())

# Acceder a un ejemplo del conjunto de entrenamiento
train_example = dataset["train"][0]
print(train_example)

# Acceder a un ejemplo del conjunto de pruebas
test_example = dataset["test"][0]
print(test_example)


In [ ]:
dataset

In [ ]:
dataset['train'][0]

In [ ]:
from datasets import Dataset, DatasetDict
import re

# Estructura de las características del conjunto de datos
features = ['pregunta', 'respuesta', 'fuente', 'tema']

# Función para limpiar el texto eliminando caracteres no deseados
def clean_text(text):
    # Eliminar caracteres extraños al final de las oraciones
    cleaned_text = re.sub(r'[,"\s]*$', '', text)
    # Eliminar el último punto y coma
    cleaned_text = cleaned_text.rstrip(';')
    return cleaned_text

# Crear el esqueleto del conjunto de entrenamiento
train_data = {feature: [] for feature in features}

# Llenar el conjunto de entrenamiento
for example in dataset['train']:
    respuesta = example.get('respuesta', '')
    respuesta_info = respuesta.split("\n")
    respuesta_texto = ''
    fuente = ''
    tema = ''
    for line in respuesta_info:
        if "respuesta" in line:
            respuesta_texto = clean_text(line.split(":", 1)[-1].strip().strip('"'))
        elif "fuente" in line:
            fuente = clean_text(line.split(":", 1)[-1].strip().strip('"'))
        elif "tema" in line:
            tema = clean_text(line.split(":", 1)[-1].strip().strip('"'))
    train_data['pregunta'].append(example['pregunta'])
    train_data['respuesta'].append(respuesta_texto)
    train_data['fuente'].append(fuente)
    train_data['tema'].append(tema)

# Asegurarnos de que todas las listas tengan la misma longitud
max_length = max(len(train_data[feature]) for feature in features)
for feature in features:
    train_data[feature] += [''] * (max_length - len(train_data[feature]))

# Crear el conjunto de datos con la estructura deseada
new_train_dataset = Dataset.from_dict(train_data)

# Repetir el proceso para el conjunto de pruebas
test_data = {feature: [] for feature in features}

for example in dataset['test']:
    respuesta = example.get('respuesta', '')
    respuesta_info = respuesta.split("\n")
    respuesta_texto = ''
    fuente = ''
    tema = ''
    for line in respuesta_info:
        if "respuesta" in line:
            respuesta_texto = clean_text(line.split(":", 1)[-1].strip().strip('"'))
        elif "fuente" in line:
            fuente = clean_text(line.split(":", 1)[-1].strip().strip('"'))
        elif "tema" in line:
            tema = clean_text(line.split(":", 1)[-1].strip().strip('"'))
    test_data['pregunta'].append(example['pregunta'])
    test_data['respuesta'].append(respuesta_texto)
    test_data['fuente'].append(fuente)
    test_data['tema'].append(tema)

# Asegurarnos de que todas las listas tengan la misma longitud
max_length = max(len(test_data[feature]) for feature in features)
for feature in features:
    test_data[feature] += [''] * (max_length - len(test_data[feature]))

new_test_dataset = Dataset.from_dict(test_data)

# Crear el nuevo conjunto de datos con la estructura deseada
new_dataset = DatasetDict({
    'train': new_train_dataset,
    'test': new_test_dataset
})

# Verificar la nueva estructura
print(new_dataset)


In [ ]:
new_dataset['train'][0]

In [ ]:
# Establecer el código de idioma
idioma = 'es_pe'

# Agregar la etiqueta de idioma a cada ejemplo en el conjunto de entrenamiento
for example in new_dataset['train']:
    example['idioma'] = idioma

# Agregar la etiqueta de idioma a cada ejemplo en el conjunto de pruebas
for example in new_dataset['test']:
    example['idioma'] = idioma

# Verificar la nueva estructura
print(new_dataset)


In [ ]:
new_dataset.push_to_hub("daqc/constitucion-politica-del-peru-1993-qa")